Trades
+ Адрес хранилища данных: points
Получение данных
Адрес хранилища данных: данные
+ download data
+ read file
Получение словаря датафремов трейдов
+ Данные: словарь датафреймов
+ method(type): type
Преобразования к таймфрему оси чарта
+ Датафрейм: Агрегированный датафрейм по таймфрему
+ Агрегация экзекьюшинов с разницей времени < 5s
+ Агрегация по таймфреймам
Рисование экзекьюшина на чарте
+ Dataframe: chart
+ plotly
главная программа
"Перенесение трейдов на график"
главная программа(передаем источник трейдов, источник таблиц для графиков, дата=последний рабочий день):
    обработка трейдов(источник трейдов, дата):
    датафрейм графика = обработка источника таблиц графика(источник таблиц графиков, дата=последний рабочий день)
    датафрейм трейдов = обработка источника хранилища (источник трейдов)
    рисуем график стака с точками входа(датафрейм графика, датафрейм трейдов)
Рисование чарта
Рисование трейдов
Candlestick
Хранилище данных: chart object
подготовка данных для графика
+ адрес хранилища данных: строка
Получение данных
+ адресс файла (строка): содержимое файла
+ чтение данных
Подготовка датафрейма
Объект данных: строка
+ csv to Dataframe
+ Convert Data
+ Prepare columns
Выбор времени
Датафрейм : рабочий датафрейм
+ обрезаем датафрейм до нужных данных
Получение таймфрейма
+ Dataframe : Dataframe
+ трансформация датафрейма 
   в нужный таймфрейм
Plotting
+ Dataframe: Plott
ploly candlestick
Чарт


деф главная программа(передаем источник трейдов, источник таблиц для графиков, дата=последний рабочий день):
    обработка трейдов(источник трейдов, дата):
    датафрейм графика = обработка источника таблиц графика(источник таблиц графиков, дата=последний рабочий день)
    датафрейм трейдов = обработка источника хранилища (источник трейдов)
    рисуем график стака с точками входа(датафрейм графика, датафрейм трейдов)

In [1]:
import pandas as pd
import re
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime as dt
import win32com.client

#plotly.tools.set_credentials_file(username='mkf', api_key='crwJntc1jJsAdaHKiDPL')

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

In [3]:
def fix_excel(trades_file):
    import win32com.client

    xl=win32com.client.Dispatch("Excel.Application")
    xl.Application.DisplayAlerts = False # disables Excel pop up message (for saving the file)
    wb = xl.Workbooks.Open(Filename=file_excel)
    wb.SaveAs(file_excel)
    wb.Close
    xl.Application.DisplayAlerts = True

In [4]:
def open_excel(path):
    try:
        open_file = pd.read_excel(path, encoding='utf-16', header=None)
    except CompDocError:
        fix_excel(path)
        open_file = pd.read_excel(path, encoding='utf-16', header=None)
    return open_file

In [5]:
def normalise_formats(storage_of_executions):
    important_columns = ['Order Id', 'Time', 'B/S', 'Qty', 'Price', 'Gross', 'Net', 'Route']
    copy_storage_of_executions = storage_of_executions.copy()
    for date, tickers in storage_of_executions.items():
        for ticker, executions in tickers.items():
            new_df = executions[important_columns]
            new_df.rename(columns={'B/S':'B_S', 'Order Id': 'Order_Id'}, inplace=True)
            new_df.insert(0, 'Date_Time', pd.to_datetime((date + ' ' + new_df['Time']), dayfirst=False))
            new_df.insert(1, 'Ticker', ticker)
            new_df['BP_Used'] = new_df['Price'] * new_df['Qty']
            new_df['Qty'] = new_df.apply(lambda x: x['Qty'] if new_df['B_S'][x.name] in ['B', 'C'] else -x['Qty'], axis=1)
            #new_df.insert(6, 'Pos_Size', new_df['Qty'].cumsum())
            new_df.reset_index(inplace=True)
            copy_storage_of_executions[date][ticker] = group_by_order_final(new_df)
    return copy_storage_of_executions

In [6]:
def group_by_order(executions):
    round_sum = lambda x: round(sum(x), 2)
    int_sum = lambda x: int(sum(x))
    executions_by_order = executions.groupby('Order_Id').agg(({'index': 'first', 'Ticker' : 'first','Time' : 'first', 
                                                               'Date_Time' : 'first', 'B_S' : 'first', 'Qty' : sum,
                                                               'Gross' : round_sum, 'Net' : round_sum, 'Route' : 'first',
                                                               'BP_Used': 'sum'}))
    
    executions_by_order['Price'] = abs(round(executions_by_order['BP_Used'] / executions_by_order['Qty'], 3))
    return executions_by_order

In [7]:
def group_by_order_final(execution):
    ''' нужен для того, чтоб ордера работали в порядке исполнения, а не только по номеру'''
    new_group_by_orger = group_by_order(pd.DataFrame(columns=execution.columns))
    one_group = pd.DataFrame(columns=execution.columns)
    for i in range(len(execution)):
        if one_group.empty:
            one_group = one_group.append(execution.iloc[i, :])
        
        else:
            if execution.iloc[i]['Order_Id'] == one_group.iloc[-1]['Order_Id']:
                one_group = one_group.append(execution.iloc[i])

            else:
                new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
                #one_group = pd.DataFrame(columns=execution.columns)
                one_group = pd.DataFrame([execution.iloc[i]])
                    
    
    new_group_by_orger = new_group_by_orger.append(group_by_order(one_group))
    new_group_by_orger.insert(6, 'Pos_Size', new_group_by_orger['Qty'].cumsum().apply(int))
    return new_group_by_orger
        

In [8]:
def preprocessing_trades(storage_of_executions):
    file_parsing = {}
    excel_df = storage_of_executions
    reindex = {a : b for a, b in zip(range(22), excel_df.iloc[2, :])}
    excel_df = excel_df.rename(columns=reindex)
    for i in range(len(excel_df)):
        if pd.notna(excel_df.iloc[i][0]):
            if re.match(r'\d{1,2}/\d{1,2}/\d{4}', excel_df.iloc[i][0]):
                date = excel_df.iloc[i][0]
                file_parsing[date] = {}
                continue

            elif re.match(r'^([\w/]+)\s{1,2}-\s.*', excel_df.iloc[i][0]):
                ticker = re.match(r'^([\w/]+)\s{1,2}-\s.*', excel_df.iloc[i][0])[1]
                file_parsing[date][ticker] = pd.DataFrame()
                continue

            elif excel_df.iloc[i][0] == 'Time':

                file_parsing[date][ticker].reindex(columns=excel_df.iloc[i])
                columns = excel_df.iloc[i]
                continue

            elif re.match(r'\d\d:\d\d:\d\d', excel_df.iloc[i][0]):
                file_parsing[date][ticker] = file_parsing[date][ticker].append(excel_df.iloc[i, :])
    date_ticker_exec = normalise_formats(file_parsing)
    return date_ticker_exec

In [9]:
#def create_points(data):
#    days_trades = preprocessing_trades(file_name)
 

In [10]:
def create_chart_df(chart_file):
    chart_df = pd.read_csv(chart_file, sep=';', decimal=',', header=0)
    chart_df[['Open', 'High', 'Low', 'Close', 'PC', 'Vol']] = chart_df[['Open', 'High', 'Low', 'Close', 'PC', 'Vol']].apply(lambda x:  round(x, 2))
    chart_df.insert(0, 'Date_Time', pd.to_datetime((chart_df['Date'] + ' ' + chart_df['Time']), dayfirst=True))
    return chart_df

In [92]:
def resampler(df):
    return df.resample('5T', {'Open': 'first', 'High' : 'max', 'Low':'min', 'Close':'last'})

In [114]:
needed_chart.iloc[220:,]

,Date_Time,Date,Time,Bar#,Bar Index,Tick Range,Open,High,Low,Close,PL,PC,MA,PH,Vol
20319,2018-08-06 12:21:00,06.08.2018,12:21:00,364/20683,-20319,4,21.27,21.29,21.24,21.24,18.3,22.1,21.23,22.45,500
20320,2018-08-06 12:20:00,06.08.2018,12:20:00,363/20683,-20320,3,21.28,21.30,21.28,21.28,18.3,22.1,21.23,22.45,3300
20321,2018-08-06 12:19:00,06.08.2018,12:19:00,362/20683,-20321,7,21.23,21.30,21.23,21.26,18.3,22.1,21.23,22.45,2060
20322,2018-08-06 12:18:00,06.08.2018,12:18:00,361/20683,-20322,16,21.31,21.32,21.16,21.16,18.3,22.1,21.22,22.45,1800
20323,2018-08-06 12:17:00,06.08.2018,12:17:00,360/20683,-20323,4,21.37,21.37,21.33,21.34,18.3,22.1,21.23,22.45,2700
20324,2018-08-06 12:16:00,06.08.2018,12:16:00,359/20683,-20324,4,21.38,21.41,21.37,21.37,18.3,22.1,21.22,22.45,900
20325,2018-08-06 12:15:00,06.08.2018,12:15:00,358/20683,-20325,4,21.42,21.42,21.38,21.38,18.3,22.1,21.21,22.45,1600
20326,2018-08-06 12:14:00,06.08.2018,12:14:00,357/20683,-20326,10,21.22,21.30,21.20,21.30,18.3,22.1,21.19,22.45,8850
20327,2018-08-06 12:13:00,06.08.2018,12:13:00,356/20683,-20327,7,21.16,21.23,21.16,21.23,18.3,22.1,21.18,22.45,3500
20328,2018-08-06 12:12:00,06.08.2018,12:12:00,355/20683,-20328,7,21.17,21.24,21.17,21.24,18.3,22.1,21.17,22.45,500


In [109]:
resampler(needed_chart.set_index('Date_Time'))
#spy_needed_chart.set_index('Date_Time')

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)



Open     21.00
High     21.09
Low      20.90
Close    21.09
Name: 2018-08-06 12:00:00, dtype: float64

In [128]:
def make_main_chart(needed_chart, drawing_trades, spy_chart):
    buy_executions = drawing_trades.loc[drawing_trades['B_S'].isin(['B', 'C'])]
    sell_executions = drawing_trades.loc[drawing_trades['B_S'].isin(['S', 'T'])]
    chart_1m = go.Candlestick(x=needed_chart.Date_Time,
                           open=needed_chart.Open,
                           high=needed_chart.High,
                           low=needed_chart.Low,
                           close=needed_chart.Close,
                          name=f"{buy_executions.Ticker[0]}_1m", 
                          decreasing = {"fillcolor": "rgb(227, 14, 0)", 
                                        "line": {
                                          "color": "rgb(8, 7, 7)", 
                                          "width": 1}},
                          increasing = {"fillcolor": "rgb(68, 171, 42)", 
                                        "line": {
                                          "color": "rgb(5, 10, 8)", 
                                          "width": 1}},
                          hoverinfo="x+y")
    chart_resampled_to5m = resampler(needed_chart.set_index('Date_Time'))
    chart_5m = go.Candlestick(x=chart_resampled_to5m.index,
                           open=chart_resampled_to5m.Open,
                           high=chart_resampled_to5m.High,
                           low=chart_resampled_to5m.Low,
                           close=chart_resampled_to5m.Close,
                          name=f"{buy_executions.Ticker[0]}_5m", 
                          decreasing = {"fillcolor": "rgb(227, 14, 0)", 
                                        "line": {
                                          "color": "rgb(8, 7, 7)", 
                                          "width": 1}},
                          increasing = {"fillcolor": "rgb(68, 171, 42)", 
                                        "line": {
                                          "color": "rgb(5, 10, 8)", 
                                          "width": 1}},
                          hoverinfo="x+y")
    
    spy_chart_candlestick = go.Candlestick(x=spy_chart.Date_Time,
                           open=spy_chart.Open,
                           high=spy_chart.High,
                           low=spy_chart.Low,
                           close=spy_chart.Close,
                          name='SPY', 
                          decreasing = {"fillcolor": "rgb(227, 14, 0)", 
                                        "line": {
                                          "color": "rgb(8, 7, 7)", 
                                          "width": 1}},
                          increasing = {"fillcolor": "rgb(68, 171, 42)", 
                                        "line": {
                                          "color": "rgb(5, 10, 8)", 
                                          "width": 1}},
                          hoverinfo="x+y", xaxis='x', yaxis='y3')
    
    text_lambda = lambda x: [f'{i.B_S}: {int(i.Qty)}, Pos_Size: {i.Pos_Size}' for i in x.itertuples()]
    
    buy_trace = go.Scatter(x=buy_executions.Date_Time, y=buy_executions.Price, mode='markers', name='Buy', 
                           marker = {"color": "rgb(103, 228, 97)", 
                                     "line": {"width": 1}, 
                                     "size": 8},
                           text=text_lambda(buy_executions),
                           hoverinfo="y+text")
    
    sell_trace = go.Scatter(x=sell_executions.Date_Time, y=sell_executions.Price, mode='markers', name='Sell',
                            marker={
                                   "color": "rgb(235, 180, 12)", 
                                   "line": {"width": 1}, 
                                   "size": 8}, 
                            yaxis= "y",
                            text=text_lambda(sell_executions),
                            hoverinfo="y+text")
    
    PH = go.Scatter(x=needed_chart.Date_Time, y=needed_chart.PH, name='PH',
                    line={"color": "rgb(0, 147, 0)", "dash": "dot", "shape": "linear", "width": 2},
                    hoverinfo='none')
    
    PL = go.Scatter(x=needed_chart.Date_Time, y=needed_chart.PL, name='PL', 
                    line={"color": "rgb(189, 103, 117)", "dash": "dot"},
                    yaxis= "y", hoverinfo='none')
    
    PC = go.Scatter(x=needed_chart.Date_Time, y=needed_chart.PC, name='PC', 
                    line={"color": "rgb(120, 75, 140)", "dash": "dash"},
                    yaxis= "y", hoverinfo='none')
    
    Vol = dict(x=needed_chart.Date_Time, y=needed_chart.Vol, type='bar', yaxis='y2',
               marker= {"color": "rgb(119, 123, 227)"}, name='Vol')
    
    data = [chart_1m, buy_trace, sell_trace, PH, PL, PC, Vol, spy_chart_candlestick, chart_5m]

    layout = {
      'title': f'{needed_chart.Date_Time.iloc[0].strftime("%Y-%m-%d")}   {drawing_trades.Ticker[0]}\
   Gross: {drawing_trades.Gross.sum()}   Vol: {int(drawing_trades.Qty.apply(abs).sum())}   Net: {drawing_trades.Net.sum()}',
      "autosize": True, 
      "bargap": 0.54, 
      "xaxis": {
        "anchor": "y2", 
        "autorange": True, 
        "domain": [0, 1], 
        "range": ["2018-08-06 05:17:30", "2018-08-06 19:47:30"], 
        "rangeslider": {
          "autorange": True, 
          "range": ["2018-08-06 05:17:30", "2018-08-06 19:47:30"], 
          #"yaxis2": {"rangemode": "match"}
        }, 
        "type": "date"
      },
        "xaxis3":{
            "anchor": "y", 
            "autorange": True, 
            "domain": [0, 0.49], 
            #"range": ["2018-08-06 05:17:30", "2018-08-06 19:47:30"], 
            "rangeslider": {
                "visible" : False,
                "autorange": True, 
              #"range": ["2018-08-06 05:17:30", "2018-08-06 19:47:30"], 
        }
      },     
        
      "yaxis": {
        "anchor": "free", 
        "autorange": True, 
        "domain": [0.17, 0.8], 
        #"overlaying": False, 
        "position": 0, 
        "range": [18.02, 23.62], 
        "type": "linear"
      }, 
      "yaxis2": {
        "anchor": "x", 
        "autorange": True, 
        "domain": [0, 0.16], 
        "range": [0, 254736.842105], 
        "type": "linear"
      },
        "yaxis3": {
        "anchor": "x", 
        "autorange": True, 
        "domain": [0.8, 1], 
        #"range": [0, 254736.842105], 
        "type": "linear"
      }
    }

    plot(dict(data=data, layout=layout), filename=f'{buy_executions.Ticker[0]}_1m.html')

In [130]:
make_main_chart(needed_chart, drawing_trades, spy_needed_chart)

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)



In [28]:
trades_file = r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls'
chart_file = r'C:\Users\User\Documents\ARLO_06.08.18.csv'
spy_file = r'C:\Users\User\Documents\SPY_example.csv'

#trades_file = r'C:\Users\Kir\Documents\07060212-2018-08-06-to-2018-08-09-detailed.xlsx'
#chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'

In [14]:
%%time
date = '8/6/2018'
ticker = 'ARLO'

#trades_file = r'C:/Users/User/Downloads/07060212-2018-08-06-to-2018-08-09-detailed.xls'
#chart_file = r'C:\Users\User\Documents\Old_comp\Chart 2016-07-16-09-54.csv'

opened_file = open_excel(trades_file)
date_tickers_exec = preprocessing_trades(opened_file)

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

E:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

Wall time: 9.1 s


In [15]:
group_by_order(date_tickers_exec[date][ticker])

,index,Ticker,Time,Date_Time,B_S,Qty,Gross,Net,Route,BP_Used,Price
Order_Id,,,,,,,,,,,
121819,3,ARLO,09:22:20,2018-08-06 09:22:20,B,200.0,0.00,-1.48,CROX,4505.76,22.529
123167,5,ARLO,09:24:03,2018-08-06 09:24:03,B,100.0,0.00,-0.76,ARCA,2231.00,22.310
128583,6,ARLO,09:31:18,2018-08-06 09:31:18,B,100.0,0.00,-0.56,NYSE,2155.00,21.550
128682,8,ARLO,09:31:47,2018-08-06 09:31:47,S,-200.0,-209.00,-210.62,EDGX,4177.00,20.885
129324,7,ARLO,09:31:41,2018-08-06 09:31:41,S,-200.0,-233.76,-235.36,EDGX,4272.00,21.360
137504,10,ARLO,09:54:53,2018-08-06 09:54:53,S,-200.0,0.00,-1.60,ARCA,4158.00,20.790
138844,11,ARLO,10:01:10,2018-08-06 10:01:10,S,-100.0,0.00,-0.81,ARCA,2087.50,20.875
138963,12,ARLO,10:27:30,2018-08-06 10:27:30,B,300.0,-226.00,-228.27,EDGX,6471.50,21.572


In [16]:
date_tickers_exec[date][ticker]

,index,Ticker,Time,Date_Time,B_S,Qty,Pos_Size,Gross,Net,Route,BP_Used,Price
Order_Id,,,,,,,,,,,,
121819,3,ARLO,09:22:20,2018-08-06 09:22:20,B,200.0,200,0.00,-1.48,CROX,4505.76,22.529
123167,5,ARLO,09:24:03,2018-08-06 09:24:03,B,100.0,300,0.00,-0.76,ARCA,2231.00,22.310
128583,6,ARLO,09:31:18,2018-08-06 09:31:18,B,100.0,400,0.00,-0.56,NYSE,2155.00,21.550
129324,7,ARLO,09:31:41,2018-08-06 09:31:41,S,-200.0,200,-233.76,-235.36,EDGX,4272.00,21.360
128682,8,ARLO,09:31:47,2018-08-06 09:31:47,S,-200.0,0,-209.00,-210.62,EDGX,4177.00,20.885
137504,10,ARLO,09:54:53,2018-08-06 09:54:53,S,-200.0,-200,0.00,-1.60,ARCA,4158.00,20.790
138844,11,ARLO,10:01:10,2018-08-06 10:01:10,S,-100.0,-300,0.00,-0.81,ARCA,2087.50,20.875
138963,12,ARLO,10:27:30,2018-08-06 10:27:30,B,300.0,0,-226.00,-228.27,EDGX,6471.50,21.572


In [17]:
drawing_trades

NameError: name 'drawing_trades' is not defined

In [18]:
drawing_trades = date_tickers_exec[date][ticker]

In [19]:
chart_df = create_chart_df(chart_file)

In [101]:
spy_chart = create_chart_df(r"C:\Users\User\Documents\SPY_example_1m.csv")

In [20]:
date_of_trade = drawing_trades.Date_Time[0]
needed_chart = chart_df[(chart_df.Date_Time.apply(lambda x: x.date()) >= date_of_trade.date()) &
      (chart_df.Date_Time.apply(lambda x: x.date()) < date_of_trade.date().replace(day=date_of_trade.day+1))]

In [102]:
spy_needed_chart = spy_chart[(spy_chart.Date_Time.apply(lambda x: x.date()) >= date_of_trade.date()) &
      (spy_chart.Date_Time.apply(lambda x: x.date()) < date_of_trade.date().replace(day=date_of_trade.day+1))]

In [103]:
make_main_chart(needed_chart, drawing_trades, spy_needed_chart)

In [ ]:
(22.54*88+22.52*112)/200


In [ ]:
ARLO	09:31:41	21.360
ARLO	09:31:47	20.885

In [ ]:
def candle_time(series, timeframe):
    #print(series, type(series))
    minute = series.minute - series.minute % timeframe
    return dt.datetime(series.year, series.month, series.day, series.hour, minute)
    
    #for i in len(dataframe):
    #    dataframe.iloc[i, 3]
    #dataframe.insert(0, 'Candle_Time', )

In [ ]:
# Раборает, но непонятно, как изменить размер сабплота
#fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True, )
#for i in data:
#    fig.append_trace(i, 1, 1)
#fig.append_trace({'x': needed_chart.Date_Time, 'y': needed_chart.Vol, 'type': 'bar', 'name':'Vol', "marker": {"color": "rgb(119, 123, 227)"}}, 2, 1)

In [ ]:
drawing_trades

In [ ]:
a = [f'{i.B_S}: {i.Qty}, \
Pos_Size: {i.Pos_Size}' for i in drawing_trades.itertuples()]

In [ ]:
a

In [ ]:
drawing_trades.rename(columns={'B/S':'B_S'})

In [ ]:
date_tickers_exec['8/6/2018']['ARLO']